In [12]:
import numpy as np
import pandas as pd
import cvxopt

In [4]:
sample = pd.read_csv("/Users/jialanma/Downloads/svm-test-datasets/2d_binary.txt",header=None)
sample

,0,1,2
0,10.028628,4.958046,1.0
1,8.915562,7.350755,1.0
2,6.661766,4.406966,1.0
3,8.964600,6.139508,1.0
4,9.288912,2.702496,1.0
...,...,...,...
195,6.164281,2.282623,1.0
196,3.747496,10.849596,-1.0
197,9.032419,6.817468,1.0
198,6.114723,2.799260,1.0


In [15]:
x = np.array(sample.iloc[:,0:2]) 
y = np.array(sample.iloc[:,2])

In [26]:
length = len(y)
length

200

In [116]:
# Split in half for training and testing data
x_train = x[:length//2]
x_test = x[length//2:]

y_train = y[:length//2]
y_test = y[length//2:]

In [117]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((100, 2), (100, 2), (100,), (100,))

In [49]:
# Build the parameters for QP solver
# H is a matrix of size (n,n) such that Hi,j=y(i)y(j)<x(i)x(j)>
row, col = x_train.shape[0], x_train.shape[0]
H = np.zeros([row,col])
for i in range(row):
    for j in range(col):
        H[i,j] = y_train[i]*y_train[j]*x_train[i,:].dot(x_train[j,:])


In [68]:
# q is a nx1 vector with -1
q = -np.ones(row)
q

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [69]:
# G is an identity matrix with -1's
G = -np.eye(row)
G

array([[-1., -0., -0., ..., -0., -0., -0.],
       [-0., -1., -0., ..., -0., -0., -0.],
       [-0., -0., -1., ..., -0., -0., -0.],
       ...,
       [-0., -0., -0., ..., -1., -0., -0.],
       [-0., -0., -0., ..., -0., -1., -0.],
       [-0., -0., -0., ..., -0., -0., -1.]])

In [60]:
# h is a nx1 vector of 0's
h = np.zeros(row)
h

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [75]:
# A is a nx1 vector of label y's
A = y_train.reshape(1,-1)
A.shape

(1, 100)

In [64]:
# b is a scalar, zero
b = 0

In [65]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [79]:
#Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(q)
G = cvxopt_matrix(G)
h = cvxopt_matrix(h)
A = cvxopt_matrix(A)
b = cvxopt_matrix(np.zeros(1))

In [80]:
#Setting solver parameters (change default to decrease tolerance) 
cvxopt_solvers.options['show_progress'] = False
cvxopt_solvers.options['abstol'] = 1e-10
cvxopt_solvers.options['reltol'] = 1e-10
cvxopt_solvers.options['feastol'] = 1e-10

In [81]:
#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

In [119]:
train_len = y_train.shape[0]
train_len

100

In [123]:
alphas = alphas.reshape(1,-1)
alphas.shape

(1, 100)

In [121]:
y_train.shape

(100,)

In [122]:
y_matrix = np.eye(train_len)
for i in range(train_len):
    y_matrix[i,i] = y_train[i]
y_matrix

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [124]:
# solve for w's and b's
lambda_y = np.dot(alphas,y_matrix)
weights = np.dot(lambda_y,x_train)
weights

array([[ 0.64362514, -1.00000044]])

In [146]:
#Selecting the set of indices S corresponding to non zero parameters
S = (alphas > 1e-4).flatten()
S

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False])

In [154]:
#Computing b
b = y_train[S].reshape(3,1) - np.dot(x_train[S], weights.reshape(-1,1))

In [155]:
b

array([[2.77022397],
       [2.77022397],
       [2.77022397]])

In [156]:
# Column mean
b_final = b.mean(0)
b_final

array([2.77022397])

In [157]:
#Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', weights.flatten())
print('b = ', b_final)

Alphas =  [0.01659302 0.69053409 0.7071271 ]
w =  [ 0.64362514 -1.00000044]
b =  [2.77022397]


In [158]:
weights.T.shape

(2, 1)

In [162]:
# Test 
y_predict = np.dot(x_test,weights.T) + b_final
for i in range(len(y_predict)):
    if y_predict[i] > 0:
        y_predict[i] = 1
    else:  # <= 0
        y_predict[i] = -1

In [163]:
y_predict[0]

array([-1.])

In [165]:
y_test[0] == y_predict[0]

array([ True])

In [166]:
count = 0
for i in range(len(y_predict)):
    if y_test[i] == y_predict[i]:
        count += 1
count 

100